In [1]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
tf.executing_eagerly()

True

In [4]:
batch_size = 32

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [5]:
from tensorflow.keras.applications import VGG16

In [6]:
model = VGG16(weights=None, classes=10, input_shape=(32,32, 3))

In [7]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator



batch_size = 32
num_classes = 10

num_predictions = 20


# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [8]:
model.load_weights('./vgg16_cifar10.h5')

In [9]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 9s 877us/sample - loss: 0.5066 - accuracy: 0.8645
Test loss: 0.5065718847632408
Test accuracy: 0.8645


In [10]:
conf = model.get_config()

In [13]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [11]:
for i, layer in enumerate(conf['layers']):
    if layer['class_name'] == 'Conv2D':
        print(i)

1
2
4
5
7
8
9
11
12
13
15
16
17


In [14]:
get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[16].output, model.layers[17].output])

In [15]:
get_output.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [16]:
x, y = get_output(x_test[0:1])

In [19]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

dataset_test = dataset.batch(batch_size)
dataset_test = dataset.repeat()

In [20]:
get_output.output[0].shape

TensorShape([None, 2, 2, 512])

In [22]:
inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
X = tf.keras.layers.SeparableConv2D(filters=get_output.output[1].shape[-1], kernel_size= (3,3), padding='Same')(inputs)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.ReLU()(X)
X = tf.keras.layers.SeparableConv2D(filters=get_output.output[1].shape[-1], kernel_size=(3,3), padding='Same')(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.ReLU()(X)
replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)

In [23]:
replacement_layers.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2, 2, 512)]       0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2_2 (Ba (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 2, 2, 512)         0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 2, 2, 512)         267264    
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 2, 2, 512)         2048      
_________________________________________________________________
re_lu_3 (ReLU)               (None, 2, 2, 512)         0   

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)

loss_object = tf.losses.MeanSquaredError()

In [25]:
replacement_layers.compile(loss=loss_object, optimizer=optimizer)

In [26]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(50000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(10000 / 32)
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [27]:
save = tf.keras.callbacks.ModelCheckpoint('./replacement_layers_2_batch_relu.h5', verbose=1, save_weights_only=True, save_best_only=True)

In [28]:
train_gen = LayerBatch(get_output, dataset)
test_gen = LayerTest(get_output, dataset_test)

In [29]:
replacement_layers.fit_generator(generator=train_gen, epochs=50, validation_data=test_gen ,verbose=0, callbacks=[save])


Epoch 00001: val_loss improved from inf to 0.00038, saving model to ./replacement_layers_2_batch_relu.h5

Epoch 00002: val_loss improved from 0.00038 to 0.00021, saving model to ./replacement_layers_2_batch_relu.h5

Epoch 00003: val_loss did not improve from 0.00021

Epoch 00004: val_loss did not improve from 0.00021

Epoch 00005: val_loss improved from 0.00021 to 0.00020, saving model to ./replacement_layers_2_batch_relu.h5

Epoch 00006: val_loss did not improve from 0.00020

Epoch 00007: val_loss improved from 0.00020 to 0.00017, saving model to ./replacement_layers_2_batch_relu.h5

Epoch 00008: val_loss did not improve from 0.00017

Epoch 00009: val_loss did not improve from 0.00017

Epoch 00010: val_loss did not improve from 0.00017

Epoch 00011: val_loss did not improve from 0.00017

Epoch 00012: val_loss improved from 0.00017 to 0.00016, saving model to ./replacement_layers_2_batch_relu.h5

Epoch 00013: val_loss did not improve from 0.00016

Epoch 00014: val_loss did not improve

In [30]:
replacement_json = replacement_layers.to_json()
with open('replacement_layers_2_batch_relu.json', 'w') as json_file:
    json_file.write(replacement_json)

In [31]:
del replacement_layers

In [32]:
with open('replacement_layers_2_batch_relu.json', 'r') as json_file:
    replacement_layers = tf.keras.models.model_from_json(json_file.read())

replacement_layers.load_weights('replacement_layers_2_batch_relu.h5')

In [38]:
replacement_layers = load_model('replacement_layers_3_batch_relu.h5')

ValueError: Unknown entry in loss dictionary: class_name. Only expected following keys: ['re_lu_3']

In [33]:
replacement_layers.compile(loss=loss_object, optimizer=optimizer)

In [34]:
replacement_layers.evaluate_generator(test_gen)

0.0017014281565934652

In [35]:
get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[16].output])

In [36]:
get_output.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [37]:
new_joint = tf.keras.Model(inputs=get_output.input, outputs=replacement_layers(get_output.output))

In [38]:
new_joint.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [39]:
bottom_half = tf.keras.Sequential()
for layer in model.layers[18::]:
    bottom_half.add(layer)

In [40]:
bottom_half.build(input_shape=new_joint.output.shape)


In [41]:
combined = tf.keras.Model(inputs=new_joint.input, outputs=bottom_half(new_joint.output))

In [42]:
combined.layers[-1].trainable=False
combined.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [38]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [43]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 300us/sample - loss: 0.5066 - accuracy: 0.8645
Test loss: 0.5065718847632408
Test accuracy: 0.8645


In [44]:
combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [45]:
del bottom_half, new_joint, replacement_layers, model

In [46]:
scores = combined.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 295us/sample - loss: 0.5035 - accuracy: 0.8641
Test loss: 0.5034622802615166
Test accuracy: 0.8641


In [47]:
combined.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=3)

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 58s 1ms/sample - loss: 0.1132 - accuracy: 0.9641 - val_loss: 0.5746 - val_accuracy: 0.8656
Epoch 2/3
50000/50000 [==============================] - 57s 1ms/sample - loss: 0.0743 - accuracy: 0.9775 - val_loss: 0.6687 - val_accuracy: 0.8688
Epoch 3/3
50000/50000 [==============================] - 57s 1ms/sample - loss: 0.0664 - accuracy: 0.9813 - val_loss: 0.7819 - val_accuracy: 0.8640


In [155]:
new_combined = tf.keras.Sequential()
new_layers = []
new_combined.add(tf.keras.layers.Input(shape=(32,32,3)))
accum = 0
for layer in combined.layers:
    if hasattr(layer, 'layers'):
        for sublayer in layer.layers:
            if(sublayer.__class__.__name__ != 'InputLayer'): 
                new_layers.append((sublayer.__class__.__name__, sublayer.get_config(), accum))
            accum += 1
    else:
        
        new_layers.append((layer.__class__.__name__, layer.get_config(), accum))
        accum += 1 



In [156]:

for i, layer in enumerate(new_layers):
    new_combined.add(keras.layers.deserialize(
                            {'class_name': layer[0], 
                             'config': layer[1]}))

In [93]:
combined.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [94]:
len(combined.layers)

19

In [157]:
accum = 0
for i, layer in enumerate(combined.layers):
    if hasattr(layer, 'layers'):
        
        for sublayer in layer.layers:
            print(f'{accum} sub is {sublayer} new is {new_combined.layers[accum]}')
            if(sublayer.__class__.__name__ != 'InputLayer'):              
                new_combined.layers[accum].set_weights(sublayer.get_weights())
                accum += 1
#             else:
#                 accum += 1
        continue
    else:
        print(layer)
        new_combined.layers[accum].set_weights(layer.get_weights())
        accum +=1 



17 sub is <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f3374698ac8> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7f2f7a720ef0>
17 sub is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7f3374698b70> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7f2f7a720ef0>
18 sub is <tensorflow.python.keras.layers.normalization.BatchNormalizationV2 object at 0x7f33644e2470> new is <tensorflow.python.keras.layers.normalization.BatchNormalizationV2 object at 0x7f2f7a740da0>
19 sub is <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f3364466ba8> new is <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f2f7a70af60>
20 sub is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7f3364466e10> new is <tensorflow.python.keras.layers.convolutional.SeparableConv2D object at 0x7f2f7a6d45f8>
21 sub is <tensorflow.python.keras.layer

In [150]:
combined.layers[18].layers

In [134]:
new_combined.build()

In [158]:
new_combined.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [159]:
scores = new_combined.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 297us/sample - loss: 0.7819 - accuracy: 0.8640
Test loss: 0.7818754182875156
Test accuracy: 0.864


In [160]:
new_combined.save('combined.h5')

In [127]:
new_combined.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 8, 8, 256)       

In [154]:
combined_json = new_combined.to_json()
with open('combined.json', 'w') as json_file:
    json_file.write(combined_json)
new_combined.save_weights('combined.h5')
#del combined
with open('combined.json', 'r') as json_file:
    json = json_file.read()
    
combined_two = tf.keras.models.model_from_json(json)
combined_two.build(input_shape=(224,224,3))

ValueError: Input 0 of layer block1_conv1 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [224, 224, 3]

In [ ]:
combined_two.layers

In [ ]:
combined.layers[2].layers